In [ ]:
:dep combee
:dep serde

In [ ]:
use combee::{read_parquet, dataframe::DataFrame};
use serde::{Serialize, Deserialize}

In [ ]:
use combee::functions::{all, avg, count};

In [ ]:
#[derive(Clone, Serialize, Deserialize, Debug)]
struct Anime {
    anime_id: u64,
    name: String,
    genre: Vec<String>,
    r#type: String,
    episodes: usize,
    rating: f64,
    members: usize
}

let df = read_parquet::<Anime>("datasets/anime.parquet".to_string()).unwrap();

println!("Count: {}", df.len());
df.head(5)

In [ ]:
// Calculating the number of rows and the average rating
df.groupby(all).agg(|_,g| (count(g), avg(g, |x| x.rating))).head(1)

In [ ]:
// Calculating the average rating for each type.
df.groupby(|x| x.r#type.clone()).agg(|r#type,g| (r#type.clone(), count(g), avg(g, |x| x.rating))).head(50)

In [ ]:
// Removing animes without a type
let df_filtered: DataFrame<Anime> = df.filter(|x| x.r#type.len() > 0);
println!("Count: {}", df_filtered.len());

df_filtered.groupby(|x| x.r#type.clone()).agg(|r#type,g| (r#type.clone(), count(g), avg(g, |x| x.rating))).head(50)

In [ ]:
// Saving result to parquet
df_filtered.to_parquet("datasets/anime_filtered.parquet".to_string())